In [139]:
import sys
  
# setting path
sys.path.append('../counterfactual')

import glob
import os
import fuse, causal_utils
from kl_general import sharpness_correction, torch_mult_fuse
glob.glob("/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/*/")

['/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_3/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_2/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_1/']

In [140]:
!ls /raid/can/nli_models/distill/nli

seed1  seed2  seed3  seed4  seed5


In [141]:
!ls /raid/can/nli_models/baseline/nli/outputs_roberta_large_clark_1_seed13370

best.th			metrics.json	      raw_pred_train.jsonl
config.json		model_state_e2_b0.th  raw_train.jsonl
counterfactual		model_state_e3_b0.th  raw_train_result_0.jsonl
distill_output_0.jsonl	model.tar.gz	      raw_train_result_1.jsonl
hans.out		normal		      temperature.pt
hans_result.jsonl	out.log		      training_state_e2_b0.th
meta.json		raw_embeddings.jsonl  training_state_e3_b0.th
metrics_epoch_0.json	raw_m.jsonl	      vocabulary
metrics_epoch_1.json	raw_mm.jsonl
metrics_epoch_2.json	raw_pred_dev.jsonl


In [142]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from scipy.special import expit

# select model
# model_path='/raid/can/nli_models/baseline/nli/outputs_roberta_large_clark_1_seed13370/'
model_path='/raid/can/nli_models/distill/nli/seed1/'

# select data path
data_path='/ist/users/canu/debias_nlu/data/nli/'
# df = pd.read_json(model_path+'raw_train.jsonl', lines=True)
# select fusion method here
fusion = fuse.sum_fuse

# Get avg  prob of bert

In [143]:
list_probs = []
for i in df['probs']:
    list_probs.extend(i)
x=np.array(list_probs)
avg=np.average(x,axis=0)

NameError: name 'df' is not defined

In [144]:
avg=np.array([1/3, 1/3, 1/3])
avg

array([0.33333333, 0.33333333, 0.33333333])

In [145]:
bias_dev_score

array([[0.17841856, 0.36916586, 0.45241557],
       [0.16769396, 0.36900166, 0.46330438],
       [0.30998087, 0.35088779, 0.33913134],
       ...,
       [0.43164769, 0.31369753, 0.25465478],
       [0.43164769, 0.31369753, 0.25465478],
       [0.19428596, 0.3688092 , 0.43690484]])

In [146]:
df_bert_dev

,pairID,gold_label,sentence1,sentence2,logits,probs
0,63735n,neutral,The new rights are nice enough,Everyone really likes the newest benefits,"[-2.1769919395, -2.075252533, 3.5961072444999997]","[0.0030898561000000002, 0.0034207623, 0.993489..."
1,91383c,contradiction,This site includes a list of all award winners...,The Government Executive articles housed on th...,"[-3.4492044449, 5.3325171471, -1.4471651316]","[0.0001533158, 0.9987114668, 0.0011351715]"
2,755e,entailment,uh i don't know i i have mixed emotions about ...,"I like him for the most part, but would still ...","[3.8359971046, -3.1393589973, -0.88918823]","[0.9902915359000001, 0.0009255596, 0.0087829642]"
3,78013c,contradiction,yeah i i think my favorite restaurant is alway...,My favorite restaurants are always at least a ...,"[-3.9847445488, 3.6016910076, 0.5973596573000001]","[0.0004830933, 0.9523091912, 0.047207769000000..."
4,96377c,contradiction,i don't know um do you do a lot of camping,I know exactly.,"[-3.069106102, 5.4561796188, -1.9531247616]","[0.0001982287, 0.9991967082000001, 0.0006051051]"
...,...,...,...,...,...,...
9995,11879n,neutral,"Since 1998, LSC has initiated and overseen sig...",LSC has been focusing on improving it's state ...,"[2.1518478394000002, -3.3257756233, 0.78730785...","[0.7938540578000001, 0.0033177144, 0.2028282583]"
9996,40715c,contradiction,Eighty percent of pagers in the United States ...,Pagers in the United States were unaffected by...,"[-2.9786710739, 3.8590564728, -0.5649821162]","[0.001058713, 0.9871100187, 0.0118312351]"
9997,4294e,entailment,"Finally, the FDA will conduct workshops, issue...",The FDA is set to conduct workshops.,"[3.7369534969, -2.345202446, -1.41116786]","[0.9919713736, 0.0022649169, 0.0057636616]"
9998,30880e,entailment,Cirque du Soleil's The latest from the acclaim...,Cirque du Soleil is an international troupe.,"[3.9056546688, -3.1757390498999998, -0.9500243...","[0.9914490581000001, 0.0008334129, 0.0077174851]"


# get score on dev set for correction

In [290]:
df_bias_dev = pd.read_json(
    data_path+'dev_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
bias_dev_score = [b for b in df_bias_dev['bias_probs']]
bias_dev_score = np.array(bias_dev_score)
# y1m0_dev = fusion(avg, bias_dev_score)
df_bert_dev = pd.read_json('/ist/users/canu/debias_nlu/data/nli/huggingface/huggingface_roberta_large_mnli_predict_dev_matched.jsonl', lines=True)
y1m1prob_dev = []
for p, h in zip(df_bert_dev['probs'], bias_dev_score):
    new_y1m1 = fusion(np.array(p), h)
    y1m1prob_dev.append(new_y1m1)
c = sharpness_correction(bias_dev_score, y1m1prob_dev) 
c = c*np.ones(3)

In [291]:
c

array([0.11455228, 0.11455228, 0.11455228])

In [292]:
df_bert_dev['probs']

0       [0.0092184423, 0.027108792200000002, 0.9636728...
1       [0.0001549819, 0.999388814, 0.0004561930000000...
2        [0.9866406322000001, 0.0007458339, 0.0126135312]
3               [0.0002055124, 0.999078393, 0.0007160408]
4        [0.0001833916, 0.9995067120000001, 0.0003098857]
                              ...                        
9995     [0.8928824067000001, 0.0005634924, 0.1065540984]
9996           [0.0001840562, 0.9994812608, 0.0003347327]
9997      [0.9879767299000001, 0.0038425154, 0.008180771]
9998    [0.9911618233, 0.0026429077, 0.006195245800000...
9999           [0.0001511162, 0.9995205402, 0.0003284238]
Name: probs, Length: 10000, dtype: object

In [293]:
CONFIG = {
    "N_LABELS": 3,
    "FUSE": torch_mult_fuse,

    "EPOCHS": 16,
    "BATCH_SIZE": 64,
    "LEARNING_RATE": 0.0001
}


# df_bias_dev = pd.read_json(
#     data_path+'dev_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
# bias_dev_score = [b for b in df_bias_dev['bias_probs']]
# bias_dev_score = np.array(bias_dev_score)
# # y1m0_dev = fusion(avg, bias_dev_score)
df_bert_dev = pd.read_json('/ist/users/canu/debias_nlu/data/nli/huggingface/huggingface_roberta_large_mnli_predict_dev_matched.jsonl', lines=True)
# y1m1prob_dev = []
# for p, h in zip(df_bert_dev['probs'], bias_dev_score):
#     new_y1m1 = fusion(np.array(p), h)
#     y1m1prob_dev.append(new_y1m1)
c2 = sharpness_correction(bias_dev_score, df_bert_dev['probs'],config=CONFIG) 
c2 = c2*np.ones(3)

In [294]:
c2

array([0.54542637, 0.54542637, 0.54542637])

#  get score from the bias model

In [295]:
# avg prob
#s score from bias model
# df_hans = pd.read_json(data_path+'hans_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
df_hans = pd.read_json(data_path+'test_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
hans_score=[b for b in df_hans['bias_probs'] ]
hans_score=np.array(hans_score)

In [296]:
!ls /ist/users/canu/debias_nlu/data/nli/

check_distill.jsonl
dev_mismatched_hard.jsonl
dev_prob_korn_full_lr_overlapping_sample_weight_3class.jsonl
dev_prob_korn_lr_overlapping_sample_weight_3class.jsonl
gam_sample_weight_clark.jsonl
gam_sample_weight_focal.jsonl
gam_sample_weight_ps.jsonl
hans_prob_korn_full_lr_overlapping_sample_weight_3class.jsonl
hans_prob_korn_lr_overlapping_sample_weight_3class.jsonl
heuristics_evaluation_set.jsonl
heuristics_evaluation_set_sample_weight.jsonl
heuristics_evaluation_set.txt
huggingface
korn_full_lr_sample_weight.jsonl
korn_lr_overlap_bias_probs.jsonl
multinli_1.0_dev_matched.jsonl
multinli_1.0_dev_matched_overlap.jsonl
multinli_1.0_dev_matched_weighted.jsonl
multinli_1.0_dev_mismatched.jsonl
multinli_1.0_dev_mismatched_overlap.jsonl
multinli_1.0_train.jsonl
multinli_1.0_train_small.jsonl
naik_stress_test
newfile.txt
README.md
small_utama_github_poe.jsonl
snli_1.0_test_hard.jsonl
test_distill.jsonl
test_kaushik_combined.jsonl
test_kaushik_RH.jsonl
test_kaushik_RP.jsonl
test_prob_korn_full

# y1m0

In [297]:
bias_score=fusion(avg,hans_score)
# bias_score=fusion(c,hans_score)
#y1m0

In [298]:
bias_score[0]

array([-0.42235353, -0.40844932, -0.41240691])

In [299]:
result_path=model_path+'normal/'
# bert model predictions on HANS
df_bert = pd.read_json('/ist/users/canu/debias_nlu/data/nli/huggingface/huggingface_roberta_large_mnli_predict_dev_mismatched.jsonl', lines=True)

In [300]:
# ent = []
y1m1prob = []
for p,h in zip(df_bert['probs'],hans_score):
    new_y1m1 = fusion(np.array(p),h)
    y1m1prob.append(new_y1m1)
  

In [301]:
softmax

<function scipy.special._logsumexp.softmax(x, axis=None)>

In [302]:
c

array([0.11455228, 0.11455228, 0.11455228])

# corrected y1m0

In [303]:
# c = sharpness_correction(hans_score, y1m1prob) 

bias_score = fusion(c,hans_score)

In [304]:
c

array([0.11455228, 0.11455228, 0.11455228])

In [305]:
y1m1prob

[array([-0.55005053, -0.23042899, -0.53774923]),
 array([-0.45911359, -0.24812024, -0.60145491]),
 array([-0.19613348, -0.5649382 , -0.60021123]),
 array([-0.57476841, -0.22905608, -0.51694023]),
 array([-0.22820652, -0.54031511, -0.55203641]),
 array([-0.25887364, -0.5282132 , -0.50572644]),
 array([-0.58386491, -0.23145939, -0.50399861]),
 array([-0.61397916, -0.52479453, -0.20808712]),
 array([-0.52135667, -0.23445421, -0.55876046]),
 array([-0.20020725, -0.57161145, -0.58321551]),
 array([-0.58907773, -0.22824345, -0.50554955]),
 array([-0.59195483, -0.52550744, -0.21706693]),
 array([-0.5748498 , -0.2281649 , -0.51862747]),
 array([-0.48470487, -0.24148405, -0.5846893 ]),
 array([-0.47273971, -0.24420394, -0.59307313]),
 array([-0.55534275, -0.22982758, -0.53379289]),
 array([-0.25609534, -0.52757765, -0.51122234]),
 array([-0.36667632, -0.5335864 , -0.36003168]),
 array([-0.5832566 , -0.52641077, -0.22047946]),
 array([-0.51998609, -0.23558462, -0.55789612]),
 array([-0.56141801,

In [306]:
bias_score

array([[-0.50324198, -0.4872655 , -0.4918163 ],
       [-0.41880719, -0.51891462, -0.55237927],
       [-0.41880719, -0.51891462, -0.55237927],
       ...,
       [-0.47644463, -0.49480525, -0.51144872],
       [-0.56194905, -0.48031786, -0.44539051],
       [-0.49571759, -0.48914247, -0.49738797]])

# TIE

In [307]:
debias_scores = []
for p,b in zip(y1m1prob,bias_score):
    debias_scores.append(p-b) # TIE  

In [308]:
df_bert

,pairID,gold_label,sentence1,sentence2,logits,probs
0,75290c,contradiction,Your contribution helped make it possible for ...,Your contributions were of no help with our st...,"[-3.0447938442, 5.6694703102, -2.3255650997]","[0.0001641442, 0.9994988441, 0.0003369633]"
1,133794c,contradiction,"The answer has nothing to do with their cause,...",Dictionaries are indeed exercises in bi-unique...,"[-2.5267326832, 4.3359084129, -1.6315977573]","[0.0010423874, 0.9964061379, 0.0025514159]"
2,3628c,entailment,We serve a classic Tuscan meal that includes ...,We serve a meal of Florentine terrine.,"[3.5418817997, -1.8530191183, -1.6873614788]","[0.9901996255000001, 0.0044951807, 0.0053050718]"
3,89411c,contradiction,"A few months ago, Carl Newton and I wrote a le...",Carl Newton and I have never had any other pre...,"[-3.5206449032, 4.5178303719, -0.7275536656]","[0.0003210049, 0.9944365025, 0.005242466]"
4,136158e,entailment,"I was on this earth you know, I've lived on th...",I don't yet know the reason why I have lived o...,"[3.4846315384, -2.787129879, -0.6260796189]","[0.9820433259, 0.0018549801, 0.016101690000000..."
...,...,...,...,...,...,...
9995,47686c,contradiction,Do you watch that?,Can you see?,"[1.3437638283, -1.7491992712000002, 0.5362580419]","[0.6705395579, 0.0304205809, 0.2990398705]"
9996,126829c,contradiction,"To a Western ear, the most predictable of lang...","To the Western ear, the least predictable of l...","[-2.8412485123, 5.2325801849, -2.0837571621]","[0.0003112842, 0.9990247488, 0.0006639436]"
9997,37368c,contradiction,The recorder captured the sounds of loud thump...,The recorder didn't capture any of the sounds.,"[-3.0385856628, 5.682038784, -2.3748254776]","[0.0001631069, 0.9995200634, 0.0003167671]"
9998,97460n,neutral,That's a good attitude!,"You feel good about this, don't you?","[1.7170525789000002, -2.9686336517000003, 1.95...","[0.4388546646, 0.0040490478, 0.5570963025]"


# Eval HANS

In [309]:
key = {0:"entailment",1:"contradiction",2:"neutral"}
t_labels = []
for i in df_bert['probs']:
    t_labels.append(key[np.argmax(i)])
df_bert['label']=t_labels

labels = []
for i in debias_scores:
    labels.append(key[np.argmax(i)])
df_bert['debias_label']=labels



In [157]:
text_ans=""
for idx, obj in enumerate(df_bert['label']):
    text_ans = text_ans + "ex"+str(idx)+","+obj+"\n"   
    
text_ans_debias=""
for idx, obj in enumerate(df_bert['debias_label']):
    text_ans_debias = text_ans_debias + "ex"+str(idx)+","+obj+"\n"          

In [246]:
def format_label(label):
    if label == "entailment":
        return "entailment"
    else:
        return "non-entailment"

guess_dict = {}
for line in text_ans.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict[parts[0]] = format_label(parts[1])
        
guess_dict_debias = {}
for line in text_ans_debias.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict_debias[parts[0]] = format_label(parts[1])        

In [160]:
labels,baseline_avg=causal_utils.get_heur(guess_dict) # normal

Heuristic entailed results:
lexical_overlap: 0.9774
subsequence: 0.9976
constituent: 0.995

Heuristic non-entailed results:
lexical_overlap: 0.2408
subsequence: 0.055
constituent: 0.1372
avg: 0.5671666666666667


In [161]:
labels,debias_avg=causal_utils.get_heur(guess_dict_debias) # TIE

Heuristic entailed results:
lexical_overlap: 0.9688
subsequence: 0.9964
constituent: 0.9902

Heuristic non-entailed results:
lexical_overlap: 0.2958
subsequence: 0.073
constituent: 0.1842
avg: 0.5847333333333333


# Causal Mediation Analysis

In [310]:
def get_ans(ans):
    if ans == 0:
        return 'entailment'
    else:
        return 'non-entailment' 
    
def get_ans(ans):
    if ans == 0:
        return 'entailment'
    elif ans ==1:
        return 'contradiction'    
    else:
        return 'neutral'
    
# y0m0
import pickle
modelname='mnli_lr_model.sav'
loaded_model = pickle.load(open(modelname, 'rb'))
x0=loaded_model.predict_proba(np.array([[0,0,0.41997876976119086]]))
m0=c
y0m0=fusion(x0,m0)    

In [311]:
labels = df_bert['gold_label']

In [312]:
factual_pred_correct = []
TIE_pred_correct = []
NIE_pred_correct = []
INTmed_pred_correct = []
pred_correct = []
all_TIE = []
all_NIE = []
all_NDE = []
all_INTmed = []
for i in range(len(labels)): 
    y1m1 = y1m1prob[i]
    y1m0 = bias_score[i]
    TE = y1m1 - y0m0
    NDE = y1m0 - y0m0
    y0m1= fusion(x0,np.array(df_bert['probs'][i]) )
    TIE = y1m1 - y1m0
    NIE = y0m1 - y0m0
    INTmed = TIE - NIE
    # factual
    factual_ans = np.argmax(df_bert['probs'][i])
    factual_ans = get_ans(factual_ans)
    factual_correct = factual_ans==labels[i]   
    factual_pred_correct.append(factual_correct)
    # TIE
    TIE_ans = np.argmax(TIE)
    TIE_ans = get_ans(TIE_ans)
    TIE_correct = TIE_ans==labels[i]  
    TIE_pred_correct.append(TIE_correct)
    # INTmed
    INTmed_ans = np.argmax(INTmed[0])
    INTmed_ans = get_ans(INTmed_ans)
    INTmed_correct = INTmed_ans==labels[i]  
    INTmed_pred_correct.append(INTmed_correct)
    # NIE
    NIE_ans = np.argmax(NIE[0])
    NIE_ans = get_ans(NIE_ans)
    NIE_correct = NIE_ans==labels[i]  
    NIE_pred_correct.append(NIE_correct)    
    
    # save
    all_NDE.append(NDE[0][0])
    all_NIE.append(NIE[0][0])
    all_TIE.append(TIE[0])
    all_INTmed.append((INTmed[0][0]))
    if  (TIE[0]/TE[0][0])<9999999:
        cf_ans = np.argmax(np.array(df_bert['probs'][i]-c2*hans_score[i]))
        cf_ans = get_ans(cf_ans)
        cf_correct = cf_ans==labels[i]
    else:
#         print(cf_ans)
        cf_correct = factual_ans ==labels[i]
    pred_correct.append(cf_correct)
    
#     np.array(df_bert['probs'][i]-bias_score)
#     labels[i]
print(np.array(all_TIE).mean(),(np.array(all_TIE).std()))
print(np.array(all_INTmed).mean(),(np.array(all_INTmed).std()))
print(np.array(all_NIE).mean(),(np.array(all_NIE).std()))

0.06128683200608272 0.13229715392350624
-0.005081695017628041 0.016050412512023667
0.06636852702371077 0.13812639597561893


In [263]:
labels[i]==factual_ans

True

'entailment'

In [252]:
NIE

array([[ 0.24263827, -0.02983795, -0.01768807]])

In [253]:
np.array(df_bert['probs'][i]-hans_score[i])

array([ 0.57260755, -0.31224327, -0.26036431])

In [254]:
df_bert['probs'][i]

[0.9017272592000001, 0.0337853096, 0.0644873977]

In [255]:
hans_score[i]

array([0.32911971, 0.34602858, 0.32485171])

In [313]:
print(sum(NIE_pred_correct)/len(labels),sum(INTmed_pred_correct)/len(labels))

0.886 0.3681


In [314]:
print(sum(factual_pred_correct)/len(labels),sum(pred_correct)/len(labels),sum(TIE_pred_correct)/len(labels))

0.8861 0.8863 0.8865


In [231]:
factual_pred_correct

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [95]:
#TIE # 0.032416620996870094 # utama best seed  0.04260506151060745
#TIE # 0.0879242620437139 # PoE
#TIE # 0.10864416858494 # Baseline 0.10295758381662952
#
# baseline
# reweight_korn
# poe_korn


In [78]:
!ls /raid/can/nli_models/

baseline	       distill		korn_overlapping  poe3
baseline_mind_distill  fake		korn_reweight	  qqp_bias
check_baseline	       fever		poe		  reweight_utama_github
clark_bias	       hypothesis_only	poe2


In [79]:
!python cma.py

In [80]:
!ls /raid/can/nli_models/baseline_mind_distill/nli

seed1  seed2  seed3


In [81]:
import cma
import glob
import numpy as np
import os
import fuse, causal_utils
model_path='/raid/can/nli_models/baseline_mind_distill/'
task='nli'
data_path='/ist/users/canu/debias_nlu/data/'
fusion = fuse.sum_fuse
test_set='mnli_hans'
correction=False

In [82]:
!ls /raid/can/nli_models/

baseline	       distill		korn_overlapping  poe3
baseline_mind_distill  fake		korn_reweight	  qqp_bias
check_baseline	       fever		poe		  reweight_utama_github
clark_bias	       hypothesis_only	poe2


In [83]:
cma.report_CMA(model_path,task,data_path,test_set,fusion,correction=correction,TIE_ratio_threshold=0.25)

TypeError: report_CMA() got an unexpected keyword argument 'TIE_ratio_threshold'

In [ ]:
!ls /raid/can/nli_models/

In [ ]:
!ls -ltr /raid/can/nli_models/baseline/nli/outputs_bert_base_clark_1_seed13370/normal/


In [ ]:
import pandas as pd

In [ ]:
  df_bias_model = pd.read_json(data_path+'test'+'_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)

In [ ]:
if '-' in df_bias_model['gold_label'].value_counts():
    print(df_bias_model['gold_label'].value_counts()['-'])

In [ ]:
import torch

In [ ]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [ ]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

In [ ]:
loss.backward()
print(w.grad)
print(b.grad)

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image
import torch.nn as nn

class CustomImageDataset(Dataset):
    def __init__(self, probs,target_probs):
        self.probs  = torch.tensor(probs)
        self.target_probs = torch.tensor(target_probs)
    def __len__(self):
        return len(self.probs)

    def __getitem__(self, idx):
        return self.probs[idx], self.target_probs[idx]


In [ ]:
df_bert['probs'][0]

In [ ]:
train=CustomImageDataset(df_bert['probs'],df_bert['probs'])

In [ ]:
class TempScale(nn.Module):
    def __init__(self):
        super(TempScale, self).__init__()
        self.T = nn.Parameter(torch.tensor(1.0))

    def forward(self, x):
        x = (x/self.T)/torch.sum(x/self.T)
        return x
model = TempScale()

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = dataloader.__len__()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
train_loop(train, model, loss_fn, optimizer)

In [ ]:
train.__len__()

In [ ]:
torch.tensor(df_bert['probs']).shape

In [ ]:
!ls ~/debias_nlu/data/nli/heuristics_evaluation_set.jsonl

In [ ]:
import pandas as pd
hans_raw = pd.read_json('/ist/users/canu/debias_nlu/data/nli/heuristics_evaluation_set.jsonl', lines=True)

In [ ]:
hans_raw


In [ ]:
hans_raw['TIE']=all_TIE
hans_raw['factual_correct']=factual_pred_correct

In [ ]:
hans_raw['factual_correct']=factual_pred_correct

In [ ]:
hans_raw.groupby(['heuristic','gold_label'])['TIE'].mean()

In [ ]:
hans_raw.groupby(['heuristic','gold_label'])['factual_correct'].mean()

In [58]:
hans_raw.groupby(['heuristic'])['TIE'].mean()

heuristic
constituent        0.119932
lexical_overlap    0.082255
subsequence        0.134800
Name: TIE, dtype: float64

In [49]:
hans_raw.groupby(['heuristic'])['factual_correct'].mean()

heuristic
constituent        0.6269
lexical_overlap    0.8296
subsequence        0.5536
Name: factual_correct, dtype: float64

In [54]:
hans_raw.groupby(['subcase','gold_label'])['TIE'].mean()

subcase                         gold_label    
ce_adverb                       entailment        0.153191
ce_after_since_clause           entailment        0.148886
ce_conjunction                  entailment        0.152524
ce_embedded_under_since         entailment        0.140753
ce_embedded_under_verb          entailment        0.122227
cn_adverb                       non-entailment    0.106354
cn_after_if_clause              non-entailment    0.133532
cn_disjunction                  non-entailment    0.125421
cn_embedded_under_if            non-entailment    0.068545
cn_embedded_under_verb          non-entailment    0.047886
le_around_prepositional_phrase  entailment        0.135009
le_around_relative_clause       entailment        0.136867
le_conjunction                  entailment        0.106735
le_passive                      entailment        0.162870
le_relative_clause              entailment        0.134555
ln_conjunction                  non-entailment    0.008906
ln_passiv

In [56]:
hans_raw.groupby(['subcase','gold_label','heuristic'])['factual_correct'].mean()

subcase                         gold_label      heuristic      
ce_adverb                       entailment      constituent        1.000
ce_after_since_clause           entailment      constituent        1.000
ce_conjunction                  entailment      constituent        1.000
ce_embedded_under_since         entailment      constituent        0.969
ce_embedded_under_verb          entailment      constituent        0.940
cn_adverb                       non-entailment  constituent        0.142
cn_after_if_clause              non-entailment  constituent        0.051
cn_disjunction                  non-entailment  constituent        0.069
cn_embedded_under_if            non-entailment  constituent        0.455
cn_embedded_under_verb          non-entailment  constituent        0.643
le_around_prepositional_phrase  entailment      lexical_overlap    0.989
le_around_relative_clause       entailment      lexical_overlap    0.985
le_conjunction                  entailment      lexical_over

In [43]:
1/0.4

2.5